# **SQL-запросы для базы данных по книгам и издательствам**

**Задача**

Составить SQL-запросы для изучения базы данных по книгам, авторам и издательствам для стартапа, запускающего сервис для чтения книг по подписке. 

База данных содержит информацию о книгах, издательствах, авторах, а также пользовательские обзоры и рейтинги книг. 

**Описание данных**
Таблица *books*:

+ book_id — идентификатор книги;
+ author_id — идентификатор автора;
+ title — название книги;
+ num_pages — количество страниц;
+ publication_date — дата публикации книги;
+ publisher_id — идентификатор издателя.

Таблица *authors*:

+ author_id — идентификатор автора;
+ author — имя автора.

Таблица *publishers*:

+ publisher_id — идентификатор издательства;
+ publisher — название издательства;

Таблица *ratings*:

+ rating_id — идентификатор оценки;
+ book_id — идентификатор книги;
+ username — имя пользователя, оставившего оценку;
+ rating — оценка книги.

Таблица *reviews*:

+ review_id — идентификатор обзора;
+ book_id — идентификатор книги;
+ username — имя автора обзора;
+ text — текст обзора.


## **Загрузка баз и создание подключения**

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

## **Запросы**

### **1. Сколько книг вышло после 1 января 2000 года**

In [3]:
# Завернем вывод результата sql-запроса в функцию 
def select(query):
  return pd.io.sql.read_sql(query, con = engine) 

In [4]:
# Выведем первые пять строк таблицы books
query = '''select *
from books t
limit 5
'''

select(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [78]:
# Посмотрим, сколько всего записей и уникальных книг содержится в базе
query = '''select count(*) as total_books, count(distinct book_id) as distinct_books
from books
'''

select(query)

,total_books,distinct_books
0,1000,1000


In [6]:
# Подсчитаем, сколько книг вышло после 1 января 2000 года
query = '''select count(distinct t.book_id) from books t
where t.publication_date >= '2000-01-01'
'''

select(query)

,count
0,821


Согласно имеющимся данным, после 1 января 2000 года вышла в свет 821 книга. Это более 80% всех книг, имеющихся в базе

### **2. Вывести название книги с максимальным кол-вом страниц**

In [47]:
query = '''select t.author_id, t.title, t.num_pages
from books t
left join authors a on t.author_id=a.author_id
where num_pages = (select max(num_pages) from books)
'''

select(query)

,author_id,title,num_pages
0,236,Harry Potter Boxed Set Books 1-5 (Harry Potte...,2690


Из всех книг в базе самой объемной по кол-ву страниц оказалась серия книг про Гарри Поттера, выпущенная одним сетом.

### **3. Составить по годам список самых объемных книг, выпущенных после  1 января 2000 года. Указать год, автора, название книги и кол-во страниц**

In [14]:
# Выведем первые пять строк таблицы authors
query = '''select * from authors t
limit 5
'''

select(query)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [36]:
# Составим список самых толстых книг, опубликованных за каждый год после 1 января 2000 года
query = '''with 
year_base as (select t.author_id, t.title, t.num_pages, date_part('year', t.publication_date)::integer as pub_year, a.author
from books t
left join authors a on t.author_id=a.author_id
where t.publication_date >= '2000-01-01'),

ranked_base as (select author, title, num_pages, pub_year, dense_rank () over (partition by pub_year order by num_pages desc) as rnk
from year_base)

select pub_year, author, title, num_pages
from ranked_base
where rnk = 1
order by pub_year
'''

select(query)

,pub_year,author,title,num_pages
0,2000,Mark Z. Danielewski,House of Leaves,705
1,2001,Diana Gabaldon,Drums of Autumn (Outlander #4),880
2,2002,Neal Stephenson,Cryptonomicon,1139
3,2003,Alexandre Dumas/Robin Buss,The Count of Monte Cristo,1276
4,2004,J.K. Rowling/Mary GrandPré,Harry Potter Boxed Set Books 1-5 (Harry Potte...,2690
5,2005,Bill Watterson,The Complete Calvin and Hobbes,1456
6,2006,Charles Dickens/Nicola Bradbury/Hablot Knight ...,Bleak House,1017
7,2007,Dan Simmons,The Terror,769
8,2008,Robin Hobb,The Mad Ship (Liveship Traders #2),906
9,2009,James Clavell,Tai-Pan (Asian Saga #2),734


### **4. Вывести авторов, книги которых были опубликованы в более чем пяти издательствах**

In [12]:
# Выведем первые пять строк таблицы publishers
query = '''select * from publishers t
limit 5
'''

select(query)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [68]:
# Выведем авторов, книги которых были опубликованы в более чем пяти издательствах
query = '''select a.author, count(distinct t.publisher_id) as publishers_cnt
from books t
left join authors a on t.author_id=a.author_id
group by a.author_id 
having count(distinct t.publisher_id) > 5
order by 2 desc
'''

select(query)

,author,publishers_cnt
0,Terry Pratchett,10
1,Stephen King,9
2,Christopher Moore,6
3,Jodi Picoult,6


### **5. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц**

In [13]:
# Найдем издательство, выпустившее наибольшее кол-во книг толще 50 стр.
query = '''with
grouped as (select t.publisher_id, p.publisher, count(distinct t.book_id) as books_cnt
from books t
left join publishers p on t.publisher_id=p.publisher_id
where t.num_pages > 50
group by t.publisher_id, p.publisher)

select t.publisher, t.books_cnt
from grouped t
where t.books_cnt = (select max(books_cnt) from grouped)
'''

select(query)

,publisher,books_cnt
0,Penguin Books,42


Наибольшее число книг толще 50 страниц вышло в издательстве Penguin Books.

### **6. Для каждой книги подсчитать количество обзоров и среднюю оценку. Отсортировать список по убыванию средней оценки и кол-ву обзоров**

In [7]:
# Выведем первые пять строк таблицы reviews
query = '''select * from reviews t
limit 5
'''

select(query)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [9]:
# Выведем первые пять строк таблицы ratings
query = '''select * from ratings t
limit 5
'''

select(query)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [109]:
# Посчитаем для каждой книги кол-во обзоров и среднюю оценку
query = '''with
rev as (select book_id, count(review_id) as reviews_cnt
from reviews
group by book_id), 

rat as (select t.book_id, avg(t.rating) as rating_avg
from ratings t
group by t.book_id)

select t.title, case when r.reviews_cnt > 0 then r.reviews_cnt else 0 end as reviews_cnt, round(rt.rating_avg, 2) as rating_avg 
from books t
left join rev r on t.book_id=r.book_id
left join rat rt on t.book_id=rt.book_id
order by rating_avg desc, reviews_cnt desc
'''

select(query)

,title,reviews_cnt,rating_avg
0,A Dirty Job (Grim Reaper #1),4,5.00
1,Moneyball: The Art of Winning an Unfair Game,3,5.00
2,School's Out—Forever (Maximum Ride #2),3,5.00
3,My Name Is Asher Lev,2,5.00
4,Wherever You Go There You Are: Mindfulness Me...,2,5.00
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,3,2.25
996,Drowning Ruth,3,2.00
997,Junky,2,2.00
998,His Excellency: George Washington,2,2.00


### **7. Вывести статистику по всей базе: среднее число обзоров на одну книгу, средний, максимальный и минимальный рейтинг**

In [72]:
# Подсчитаем для всех книг среднее число обзоров, среднюю, максимальную и минимальную оценку
query = '''with
rev as (select book_id, count(review_id) as reviews_cnt
from reviews
group by book_id), 

rat as (select t.book_id, avg(t.rating) as rating_avg
from ratings t
group by t.book_id),

sum_table as (select t.book_id, t.title, r.reviews_cnt, rt.rating_avg
from books t
left join rev r on t.book_id=r.book_id
left join rat rt on t.book_id=rt.book_id)

select 'Среднее число обзоров на книгу' as description, round(avg(reviews_cnt), 2) as stats from sum_table
union all
select 'Средняя оценка', round(avg(rating_avg), 2) from sum_table
union all
select 'Максимальная оценка', max(rating_avg) from sum_table
union all
select 'Минимальная оценка', min(rating_avg) from sum_table
'''

select(query)

,description,stats
0,Среднее число обзоров на книгу,2.81
1,Средняя оценка,3.90
2,Максимальная оценка,5.00
3,Минимальная оценка,1.50


В среднем на каждую книгу приходится 2,8 обзоров. Средняя оценка по всем книгам - 3.9, максимальная  оценка - 5, минимальная - 1,5.

### **8. Определить автора с самой высокой средней оценкой книг. Учитывать только книги с 50 и более оценками**

In [15]:
# Найдем автора с самой высокой средней оценкой книг, учитывая книги с 50 и более оценками
query = '''with 
rat as (select t.book_id, avg(t.rating) as rating_avg, count(distinct t.rating_id) as marks_cnt
from ratings t
group by t.book_id
having count(distinct t.rating_id) >= 50)

select a.author, round(t.rating_avg, 2), t.marks_cnt
from rat t
left join books b on t.book_id=b.book_id
left join authors a on b.author_id=a.author_id
where t.rating_avg = (select max(rating_avg) from rat)
'''

select(query)

,author,round,marks_cnt
0,J.K. Rowling/Mary GrandPré,4.41,82


Автор с самой высокой средней оценкой книг - Дж.Роулинг, чьи книги иллюстрировала Мэри Гранпре. Средняя оценка ее книг - 4,4. Всего в базе по этому автору есть данные о 82 оценках.



### **9. Подсчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок**

In [16]:
query = '''with
users as (select t.username
from ratings t
group by t.username
having count(distinct t.rating_id) > 50),

reviews_cnt as (select t.username, count(distinct t.review_id)
from reviews t
where t.username in (select * from users)
group by t.username)

select round(avg(count), 1) as rev_avg
from reviews_cnt
'''

select(query)

,rev_avg
0,24.3


Пользователи, которые поставили больше 50 оценок, в среднем сделали 24 обзора. 

### **10. Вывести список самых активных пользователей, которые входят в тройку лидеров по кол-ву обзоров или в тройку лидеров по кол-ву простравленных оценок**

In [107]:
# Выведем первые пять строк таблицы reviews
query = '''with
reviews_cnt as (select username, count(review_id) as rev_cnt, dense_rank() over (order by count(review_id) desc) as rev_rnk
from reviews 
group by username), 

rating_cnt as (select username, count(rating_id) as rat_cnt, dense_rank() over (order by count(rating_id) desc) as rat_rnk
from ratings
group by username)

select username as most_active_users
from reviews_cnt
where rev_rnk <= 3
union
select username
from rating_cnt
where rat_rnk <= 3
'''

select(query)

,most_active_users
0,paul88
1,susan85
2,martinadam
3,sfitzgerald
4,jennifermiller
5,richard89
